In [5]:
from generic_model import Model
from keras import optimizers
import keras.backend as KB
import keras.losses as Loss
import tensorflow as tf
import os
import sys

In [6]:
ROOTDIR=os.path.abspath("../")
label_encoder_dir=os.path.join(ROOTDIR,"Preprocessing")
data_dir=os.path.join(ROOTDIR,"Data")
sys.path.append(ROOTDIR)

In [7]:
from Preprocessing.Preprocessor import preprocessor
from data_utils.keras_generator import Generator
from utils.losses import weighted_cross_entropy_loss,f1_score_metric
from utils.callbacks import get_tensorboard_callback,get_checkpoint_call_back

In [8]:
import pandas as pd
files=os.listdir(data_dir)
array=pd.read_hdf(os.path.join(data_dir,files[0]))
P=preprocessor(label_encoder_dir)
array=P.process_batch(array)
action_col=array.columns=='action'
g_config={'labels':6,
        'weightage':[1,1,1,1,1,1],
        'data_directory':data_dir,
        'file_batch_size':1,
        'format_':'h5',
        'ex_batch_size':10000,
        'file_examples':5000,
        'max_queue_size':10,
        'Preprocessor':P,
        'static_weightage':False,
         'action_col':action_col}

G=Generator(config=g_config)


C:\ProgramData\Anaconda2\envs\py3\lib\site-packages\sklearn\base.py:311: UserWarning: Trying to unpickle estimator LabelEncoder from version 0.20.1 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [11]:
nn_config={'layer_shapes':[200,200,200,200],'Activation':'relu','Output':6,'Input_shape':472, 'model_type':'fc'}
my_model=Model(nn_config) 
f1=f1_score_metric(6)
my_model.model.compile(loss=weighted_cross_entropy_loss, optimizer=optimizers.Adam(lr=0.0001),metrics=["acc",f1])

In [10]:
from multiprocessing import Process
p1 = Process(target=G.generate)
p1.start()

In [ ]:
my_model.model.fit_generator(generator= G, 
                    steps_per_epoch  = len(G), 
                    epochs           = 10, 
                    verbose          = 1,
#                     validation_data  = valid_batch,
#                     validation_steps = len(valid_batch),
                    max_queue_size   = 3)

Epoch 1/10


In [12]:
print (G.Data.qsize())

1
